In [3]:
import pandas as pd 
import glob
from datetime import datetime 

from bs4 import BeautifulSoup

In [4]:
INPUT_DATA_FOLDER = "data/0_isw_data_collection/0_raw_isw_data"

#OUTPUT_FOlDER = "data/1_isw"
#OUTPUT_DATA_FILE = "all_days_isw_reports.csv"

In [5]:
files_by_days = glob.glob(f"{INPUT_DATA_FOLDER}/*.html")

In [6]:
len(files_by_days)

print(files_by_days[0])


data/isw_data_collection/0_raw_isw_data/18_7_2022___RusCampaignJuly18.html


In [7]:
all_data = []

for file in files_by_days:
    
    d = {}
    file_name = file.split("/")[-1].split("___")
    date = datetime.strptime(file_name[0],"%d_%m_%Y")
    url = file_name[1].split(".")[0]
    
    with open(file,"r") as cfile:
        parsed_html = BeautifulSoup(cfile.read())
        
        title = parsed_html.head.find("title").text
        link = parsed_html.head.find("link",attrs={"rel":"canonical"},href=True)
        if parsed_html.head.find("link",attrs={"rel":"canonical"},href=True) is not None:
            link = parsed_html.head.find("link",attrs={"rel":"canonical"},href=True).attrs["href"]
        if parsed_html.body.find('h1', {'class': 'title', 'id': 'page-title'}) is not None:
            text_title = parsed_html.body.find('h1', {'class': 'title', 'id': 'page-title'}).text
        text_main = parsed_html.body.find('div',attrs={"class":"field field-name-body field-type-text-with-summary field-label-hidden"})
    #    print( parsed_html.body.find('h1', {'class': 'title', 'id': 'page-title'}).text)
        
        d = {
            'date': date,
            'short_url': url,
            'title': title,
            'text_title': text_title,
            'full_url': link,
            'main_html': text_main
        }
         
        all_data.append(d)

In [8]:
data = pd.DataFrame(all_data)

In [9]:
data['main_html'][1]

<div class="field field-name-body field-type-text-with-summary field-label-hidden"><div class="field-items"><div class="field-item even" property="content:encoded"> <p align="center" style="text-align: left;"> </p>
<p align="center" style="text-align: left;"><strong>Mason Clark, George Barros, and Karolina Hird</strong></p>
<p align="center" style="text-align: left;"><strong>April 2, 5:00 pm ET</strong></p>
<p style="text-align: left;"><strong style="font-size: 13.008px;">Continuing Russian operations along their new main effort in eastern Ukraine made little progress on April 2, and Russian forces likely require some time to redeploy and integrate reinforcements from other axes.</strong><span style="font-size: 13.008px;"> Ukrainian forces repelled likely large-scale Russian assaults in Donbas on April 2 and inflicted heavy casualties. Russian forces continued to capture territory in central Mariupol and will likely capture the city in the coming days. Russian units around Kyiv and in 

In [10]:
data = data.sort_values(by=['date'])

In [11]:
import requests

url = 'https://understandingwar.org/backgrounder/russian-offensive-campaign-assessment-march-9-2023'
response = requests.get(url)

html_code = response.content.decode('utf-8') #decode the content into a string

print(html_code)


<!DOCTYPE html>
<html xmlns:og="http://opengraphprotocol.org/schema/" lang="en" dir="ltr">

<head>
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta name="Generator" content="Drupal 7 (http://drupal.org)" />
<link rel="canonical" href="/backgrounder/russian-offensive-campaign-assessment-march-9-2023" />
<link rel="shortlink" href="/node/5101" />
<link rel="shortcut icon" href="https://understandingwar.org/sites/default/themes/isw/isw.ico" type="image/vnd.microsoft.icon" />
<meta name="keywords" content="Institute for the Study of War, ISW, War, Institute, Study, ORBAT, Report, Iraq, Iran, Afghanistan, Middle East, Libya, Security,,Institute for the Study of War, ISW, War, Institute, Study, ORBAT, Report, Iraq, Iran, Afghanistan, Middle East, Libya, Security," />
<meta name="description" content="Russian forces conducted the largest missile strike across Ukraine of 2023 so far on March 9, but the attack likely only served Russian state propaganda objectives. 

In [12]:
data.head()

,date,short_url,title,text_title,full_url,main_html
277,2022-02-25,RusCampaignFeb25,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"[[[ , <p align=""center"" style=""text-align: lef..."
323,2022-02-26,RusCampaignFeb26,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"[[[ , <p align=""center"" style=""text-align: lef..."
113,2022-02-27,RusCampaignFeb27,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"[[[ , <p style=""text-align: left;""><strong><sp..."
155,2022-02-28,RusCampaignFeb28,"Russian Offensive Campaign Assessment, Februar...","Russian Offensive Campaign Assessment, Februar...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef..."
200,2022-03-01,RusCampaignMar1,"Russian Offensive Campaign Assessment, March 1...","Russian Offensive Campaign Assessment, March 1",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef..."


In [13]:
data.shape

(340, 6)

In [14]:
data.isnull().count()

date          340
short_url     340
title         340
text_title    340
full_url      340
main_html     340
dtype: int64

In [15]:
data.to_csv('data/0_isw_data_collection/0_raw_isw_parsed.csv')